In [171]:
import nbformat
from casstools.path_parser import PathParser
from casstools import code_tools as ct

OK = "\U00002705"
QUESTION = "\U00002753"
CANCEL = "\U0000274C"


class NotebookFile(object):

    def __init__(self, filespec = None):
        
        if filespec is None:
            filespec = PathParser().fullpath

        self.filespec = filespec
        self.contents = self.load_notebook(filespec)
        self.filetype = self.__get_file_type(filespec)


    def __get_file_type(self, filespec):
        file_type = filespec.split("/")[-1].split("-")[0].upper()
        return file_type

    def load_notebook(self, path: str):
        with open(path, "r") as f:
            return nbformat.read(f, as_version=4)

    def _extract_metadata_value(self, cell, key):
        '''
        Extracts metadata value from cell using key 
        first checks under ['metadata']['cass'][key]
        then under legaccy ['metadata'][key]
        '''
        md = cell['metadata']
        if md.get("cass", None) is not None:
            return md['cass'].get(key, None)
        else:
            return md.get(key, None)

    @property
    def code_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "code"]

    @property
    def markdown_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "markdown"]

    @property
    def raw_cells(self):
        return [c for c in self.contents.cells if c.cell_type == "raw"]


    def has_submission_cell(self):
        submission = "from casstools.assignment import Assignment\nAssignment().submit()"
        for c in self.code_cells:
            if c['source'].find(submission) >=0:
                return True
        return False
            

    def code_cells_of_type(self, cell_type, executed_only=False):
        '''
        return a list of code cells with the "cell_type" as their designation
        designation is found in:
        1) ['metadata']['cass']['code_cell_type'] or 
        2) ['metadata']['code_cell_type'] if the "cass" key does not exist.
        '''
        cells = [c for c in self.code_cells if self._extract_metadata_value(c, "code_cell_type") == cell_type]
        if executed_only:
            return [c for c in cells if c.execution_count is not None and c.execution_count >=1]
        else:
            return cells

    def markdown_cells_of_type(self, cell_type):
        cells = [c for c in self.markdown_cells if self._extract_metadata_value(c, "label") == cell_type]
        return cells 
        
    @property
    def exercise_code_cells(self):
        cells = [c for c in self.code_cells if self._extract_metadata_value(c, "code_cell_type") in ["debug_code", "write_code"]]
        return cells

    def __check(self, boolean_expression):
        if boolean_expression:
            return OK
        else:
            return CANCEL

    def validate_lab(self):
        '''
        validated the lab file has the necessary metadata cells for the self-check, and lab is ready to publish. This is an author's tool to make sure check_lab will work.
        '''
        check1 = self.__check(self.has_submission_cell)
        print(f"\t{check1} notebook: has submission cell")
        
        for rc_cell in self.code_cells_of_type("run_code"):

            print("CELL:", rc_cell)
            
            check1 = self.__check(len(rc_cell['source']) > 0)
            print(f"\t{check1} run_code cell: should have has code")
            
            check2 = self.__check(rc_cell['execution_count'] is None)
            print(f"\t{check2} run_code cell: should be no execution_count")                

        for ec_cell in self.exercise_code_cells:

            print("CELL", ec_cell)

            label = self._extract_metadata_value(ec_cell, "label")
            check1 = self.__check(label is not None)           
            print(f"\t{check1} exercise_code cell: has label {label}")

            code_cell_type = self._extract_metadata_value(ec_cell, "code_cell_type")
            check2 = self.__check(code_cell_type is not None)           
            print(f"\t{check2} exercise_code cell: has code_cell_type {code_cell_type}")

            if code_cell_type == "write_code":
                student_code = "\n".join([line for line in ec_cell.source.split("\n") if not line.strip().startswith("#")]).strip()
                check3 = self.__check(student_code == "")
                print(f"\t{check3} exercise_code cell: code_cell_type='write_code' cell should be empty {student_code}")

            solution_code = "".join(self._extract_metadata_value(ec_cell, "solution"))
            check4 = self.__check(solution_code is not None)
            print(f"\t{check4} exercise_code cell: should have solution_code metadata")

        comfort_cells = self.markdown_cells_of_type("comfort_cell")
        print("CELL", comfort_cells)
        check1 = self.__check(len(comfort_cells)==1)       
        print(f"\t{check1} comfort_cell: should have ONE comfort cell")
        check2 = self.__check(comfort_cells[0]['source'].strip()=="")
        print(f"\t{check1} comfort_cell: should should be empty")

        questions_cells = self.markdown_cells_of_type("question_cell")
        print("CELL", questions_cells)
        check1 = self.__check(len(questions_cells)==1)       
        print(f"\t{check1} question_cell: should have ONE questions cell")
        check2 = self.__check(questions_cells[0]['source'].strip()=="")
        print(f"\t{check1} question_cell: should should be empty")
        

    
    def check_lab(self, output_issues=True):
        '''
        Pre-check/ pre-grade lab before submission.
        '''
        row = { 'issues': [], 'details': [] }
        # INVENTORY
        run_code_cells = self.code_cells_of_type("run_code")
        exercise_code_cells =  self.exercise_code_cells
        all_code_cells = run_code_cells + exercise_code_cells
        executed_cells = [c for c in all_code_cells if c.execution_count is not None and c.execution_count >=1]
        row['code_cells'] = len(all_code_cells)
        row['code_cells_executed'] = len(executed_cells)
        row['code_cells_pct'] = f"{row['code_cells_executed']/row['code_cells']}"
        if float(row['code_cells_pct']) < 1.0:
            row['issues'].append("Not all code cells were executed.")
            
        try:
            comfort_cell = self.markdown_cells_of_type("comfort_cell")[0]
            row['comfort_level'] = comfort_cell['source'].strip()
            if row['comfort_level'] == "":
                row['issues'].append("Comfort level is blank.")
            elif row['comfort_level'].isdigit():
                c = int(row['comfort_level']) 
                if not c in [1,2,3,4]:
                    row['issues'].append("Comfort level should be 1,2,3 or 4.")
            else:
                 row['issues'].append("Comfort level should be 1,2,3 or 4.")
                
        except IndexError:
            print("ERROR: Missing Comfort Cell. Did you erase it?")

        try:
            question_cell = self.markdown_cells_of_type("question_cell")[0]
            row['questions'] = question_cell['source'].strip()
            if row['questions'] == "":
                row['issues'].append("Questions cell is blank. You should have a question or comment.")
    
        except IndextError:
            print("ERROR: Missing Question Cell. Did you erase it?")

        
        for cell in exercise_code_cells:
            label = self._extract_metadata_value(cell, "label")
            student_code = "\n".join([line for line in cell.source.split("\n") if not line.strip().startswith("#")]).strip()
            solution_code = "".join(self._extract_metadata_value(cell, "solution"))
            syntax = ct.syntax_check(student_code)
            similarity = ct.code_similarity_check(solution_code, student_code)

            row[label] = { 
                'has_code': 'no' if student_code == "" else 'yes',
                'syntax': 'ok' if syntax['ok'] else 'error',
                'similarity': similarity['pct_similar']
            }
            details = {
                'label': label,
                'data' : row[label],
                'syntax': syntax,
                'similarity': similarity,
                'student': student_code,
                'solution': solution_code
            }
            row['details'].append(details)

            if row[label]['has_code'] == 'no':
                row['issues'].append(f"{label} does not have a code solution.")
            elif row[label]['syntax'] == 'error':
                row['issues'].append(f"{label} code has syntax error: {syntax['error']}")
            elif float(row[label]['similarity']) < 0.5:
                row['issues'].append(f"{label} code not at least 50% similar to expected solution.")

        
        if output_issues:
            for issue in row['issues']:
                print(f"{CANCEL} {issue}")
            if len(row['issues']) == 0:    
                print(f"{OK} The lab submission appears to have no issues.")
                print(f"  {row['code_cells_pct']} Percent of cell executed.")
                print("  Summary of code Exercises")
                print("  CODE\tSYNTAX\tSIMILARITY")
                for d in row['details']:
                    print(f"  {d['label']}\t{d['data']['syntax']}\t{d['data']['similarity']}")
        else:         
            return row


In [172]:
PathParser().fullpath
nb = NotebookFile()
nb.filespec

'/home/jovyan/library/git/casstools/notebooks/Sample.ipynb'

In [173]:
nb = NotebookFile("/home/jovyan/library/ist256/spring2024/lessons/01-Intro/LAB-Intro.ipynb")
nb.check_lab()
# nb.code_cells

❌ Not all code cells were executed.
❌ Comfort level is blank.
❌ Questions cell is blank. You should have a question or comment.
❌ 1.1 does not have a code solution.
❌ 1.2 code has syntax error:    line 1
    name = input "Enter your name: "
                 ^^^^^^^^^^^^^^^^^^^
SyntaxError: invalid syntax

❌ 1.3 does not have a code solution.
❌ 1.4 does not have a code solution.
❌ 1.5 does not have a code solution.


In [170]:
nb = NotebookFile("/home/jovyan/library/ist256/spring2024/lessons/01-Intro/LAB-Intro.ipynb")
total_run_code_cells = len(nb.code_cells_of_type("run_code"))
total_executed_run_code_cells = len(nb.code_cells_of_type("run_code", executed_only=True))
print(nb.filetype)

LAB


In [8]:
for cell in nb.exercise_code_cells:
    label = nb._extract_metadata_value(cell, "label")
    student_code = "\n".join([line for line in cell.source.split("\n") if not line.strip().startswith("#")])
    solution_code = "".join(nb._extract_metadata_value(cell, "solution"))
    syntax = ct.syntax_check(student_code)
    similarity = ct.code_similarity_check(solution_code, student_code)
    print(label, syntax, similarity)
    print(student_code, solution_code)

1.1 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 5, 'pct_similar': 0.0}
 x = input("What is your name? ")
print("Hello there",x)

1.2 {'ok': False, 'output': '', 'error': '   line 1\n    name = input "Enter your name: "\n                 ^^^^^^^^^^^^^^^^^^^\nSyntaxError: invalid syntax\n'} {'similar_token_count': 0, 'total_tokens': 0, 'pct_similar': 0}
name = input "Enter your name: "
foo = input("Enter your age: ")
print(name, "is" )
 name = input("Enter your name: ")
age = input("Enter your age: ")
print(name, "is", age)

1.3 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 6, 'pct_similar': 0.0}
 first_name = input("Enter your first name:")
last_name = input("Enter your last name:")
print("Hello",first_name, last_name)

1.4 {'ok': True, 'output': '', 'error': ''} {'similar_token_count': 0, 'total_tokens': 19, 'pct_similar': 0.0}
 first=input("What is your favorite color?")
second=input("What is your second favor

In [6]:
syntax

{'ok': True, 'output': '', 'error': ''}

TypeError: object of type 'NoneType' has no len()

In [1]:
from assignment import Assignment
Assignment()._confirm_submission(False,True)

ℹ Submit Again? [y/n] ❓  ds


False

In [3]:
from datetime import datetime

In [17]:
assignment_due_date = "2024-09-20 23:59"
due = datetime.strptime(assignment_due_date, "%Y-%m-%d %H:%M")
now = datetime.now()

In [15]:
now.strftime("%Y-%m-%d %H:%M")

'2024-01-06 12:34'

In [21]:
now <= due

True

In [2]:
import pandas as pd

In [12]:
pd.read_json("data/demo.json", lines=True, orient="values")

,name,age
0,abby,10
1,bob,11
2,chris,12


In [5]:
from assignment import Assignment
Assignment().read_submission_log()

,TIMESTAMP,COURSE,TERM,USER,STUDENT,PATH,ASSIGNMENT,POINTS,DUE DATE,LATE,STATUS,RECIEPT
0,2024-01-06 13:14:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
1,2024-01-06 13:15:00,ist256,spring2024,mafudge@syr.edu,True,ist256/spring2024/00-testing/testing.ipynb,testing.ipynb,5,2024-09-20 23:59,False,On Time,28158a2cb4b3c23fc46b7a169b287c3c
